The Siemens PSS/E Format is widely used by electrical grid operators around the United States, and is meant to be used with their software (which is $7,600). There seems to be no open-source code to translate between this data to a unified grid model that can be translated to other data formats.

See the files in `data/WECC_240bus*` folder. 
The `.raw` file contains all of the grid information including a load snapshot, 
and the `dyr` contains the information for transient stability modelling, [whatever that is](https://www.powerworld.com/WebHelp/Content/MainDocumentation_HTML/Transient_Stability_Data_GEDYD.htm).


In [ ]:
#TODO: store in pandas dataframe instead of dictionary of lists?
import pandas as pd

def PSS_raw_reader(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    section = 'system'
    ret = ''
    i = 0
    DATA = {}
    sections = ['bus', 'load', 'fixed shunt','generator','branch','system switching device','transformer','area','two-terminal dc','vsc dc','impedance correction','multi-terminal dc','multi-section line','zone','inter-area transfer','owner','facts device','switched shunt','gne','induction machine','substation']
    while i < len(lines):
        line = lines[i]
        # SECTION CHANGE
        if line.startswith("0 / "):
            ret = ''
            msgs = line[4:].split(',')
            old = msgs[0][7:]  # skip "END OF "
            print(old+'\n'+ret)
            if len(msgs) != 2:
                # we have reached the end
                break
            new = msgs[1][7:].lower() # skip " BEGIN "
            section = new[:-5]  # skip " DATA"
            titles = []
            i += 1
            continue
        # column titles
        elif line.startswith('@!'):
            titles = line[2:].lower().split(',')
            i += 1
            continue
        # GET DATA
        if section == 'system':
            ret += line + '\n'
        else:
            #TODO: 
            DATA[section] = DATA.get(section, pd.DataFrame)
            # PARSE ROW OF DATA
            parts = line.split(',')
            for pidx in range(len(parts)):
                try:
                    parts[pidx] = int(parts[pidx])
                except:
                    try:
                        parts[pidx] = float(parts[pidx])
                    except:
                        # strings are encoded as 'STR     '
                        parts[pidx] = parts[pidx][1:-1].rstrip()
            if titles:
                assert len(titles) == len(parts)
                entry = {titles[j]: parts[j] for j in range(len(parts))}

            if section == 'bus':
                DATA[section][]
            # index branches by (I,J)
            elif section == 'branch':
                DATA[section] = DATA.get(section, {})

            
        

        i += 1

